# Teoría de Markowitz

La teoría moderna de portafolios (Modern Portfolio Theory, MPT) desarrollada por Harry Markowitz (1952) se basa en las siguientes ideas y supuestos básicos:

- **Objetivo**: seleccionar un portafolio de activos que, para un nivel dado de riesgo, maximice el rendimiento esperado (o, equivalente, minimice el riesgo para un rendimiento esperado dado).
- **Riesgo**: se mide por la varianza (o desviación estándar) del rendimiento del portafolio. Se asume que los inversores son aversos al riesgo y que sólo les importan la media y la varianza del rendimiento (preferencia cuadrática o rendimientos normales).
- **Información**: los inversores conocen las medias, varianzas y covarianzas de los rendimientos de los activos (o pueden estimarlas).
- **Mercado**: no se consideran fricciones (no hay costos de transacción, impuestos), se permiten posiciones largas y, si se desea, cortas; los activos son divisibles.



## 1. Especificación del problema de optimización

Sea un universo de $n$ activos. Definimos:

- $\mu$ : vector columna $(n\times 1)$ de rendimientos esperados de cada activo.
- $\Sigma$ : matriz de covarianza $(n\times n)$ de los rendimientos.
- $w$ : vector de pesos del portafolio $(n\times 1)$, con la restricción $\mathbf{1}^T w = 1$ (donde $\mathbf{1}$ es el vector de unos).

El rendimiento esperado del portafolio:

$$E[R_p] = w^T \mu$$

La varianza del portafolio:

$$\text{Var}(R_p) = w^T \Sigma w$$

La desviación estándar (riesgo) es $\sigma_p = \sqrt{w^T \Sigma w}$.

Problema:

$$\begin{aligned}
&\underset{w}{\text{minimizar}} && w^T \Sigma w \\
&\text{sujeto a} && w^T \mu = \mu_p \\
& && \mathbf{1}^T w = 1
\end{aligned}$$

Donde $\mu_p$ es el rendimiento esperado objetivo del portafolio.

Se puede resolver mediante multiplicadores de Lagrange. Construimos:

$$\mathcal{L} = w^T \Sigma w - \lambda (w^T \mu - \mu_p) - \gamma (\mathbf{1}^T w - 1)$$

Derivando respecto a $w$ e igualando a cero:

$$2\Sigma w - \lambda \mu - \gamma \mathbf{1} = 0$$

Por lo tanto:

$$w = \frac{1}{2} \Sigma^{-1} (\lambda \mu + \gamma \mathbf{1})$$

Usando las restricciones se obtienen sistemas lineales que conducen a las constantes. Definimos los escalares clásicos:

$$A = \mathbf{1}^T \Sigma^{-1} \mathbf{1}, \quad B = \mathbf{1}^T \Sigma^{-1} \mu, \quad C = \mu^T \Sigma^{-1} \mu$$

y el determinante:

$$D = AC - B^2$$

Entonces los multiplicadores se resuelven y el vector de pesos para un objetivo $\mu_p$ es:

$$w^*(\mu_p) = \Sigma^{-1} \left(\frac{C - B \mu_p}{D} \mathbf{1} + \frac{A \mu_p - B}{D} \mu \right)$$

Esto genera la **frontera eficiente** al variar $\mu_p$.


## 2. Portafolio de mínima varianza global (GMV)

El portafolio de mínima varianza global se obtiene al resolver la optimización sin la restricción del rendimiento esperado (sólo suma de pesos = 1):

$$\underset{w}{\text{minimizar}} \quad w^T \Sigma w \quad \text{sujeto a} \quad \mathbf{1}^T w = 1$$

La solución cerrada es:

$$w_{GMV} = \frac{\Sigma^{-1} \mathbf{1}}{\mathbf{1}^T \Sigma^{-1} \mathbf{1}} = \frac{\Sigma^{-1} \mathbf{1}}{A}$$

Su varianza mínima es $\sigma_{GMV}^2 = 1/A$.

## 3. Portafolio tangente (con activo libre de riesgo)

Si existe una tasa libre de riesgo $r_f$, la mejor combinación de activos riesgosos para un inversor que puede invertir en el activo libre de riesgo es el **portafolio tangente** (o portafolio de máxima Sharpe).

El vector de pesos del portafolio tangente (sin imponer suma 1; luego se normaliza) se obtiene maximizando la razón de Sharpe:

$$\max_w \frac{w^T (\mu - r_f \mathbf{1})}{\sqrt{w^T \Sigma w}}$$

La solución (antes de normalizar) es proporcional a:

$$w^* \propto \Sigma^{-1} (\mu - r_f \mathbf{1})$$

Normalizando para que los pesos sumen 1:

$$w_{T} = \frac{\Sigma^{-1} (\mu - r_f \mathbf{1})}{\mathbf{1}^T \Sigma^{-1} (\mu - r_f \mathbf{1})}$$

La **recta del mercado de capitales** (CML) muestra cómo combinar el activo libre de riesgo con el portafolio tangente para obtener la frontera eficiente con libre de riesgo.


## 4. Interpretación

- La matriz inversa de covarianza, $\Sigma^{-1}$, actúa como un operador que pondera las exposiciones relativas según riesgos y correlaciones.
- Si un activo está altamente correlacionado con otros o tiene gran varianza, su peso óptimo se verá reducido, todo lo demás constante.
- La frontera eficiente es parabólica en el espacio (riesgo, rendimiento). Cada punto representa un portafolio "óptimo" para un nivel de retorno.


# Implementacion

In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
# -------------------------------------------------
# 3. Función para obtener los estimadores muestrales de los retornos
#    y matriz de varianza
# -------------------------------------------------
def valores_muestrales(stocks: pd.DataFrame):
    m, n = stocks.shape
    retornos = stocks.sum(axis=0) / m
    
    # Matriz de covarianza (estimada manualmente, como en R)
    matriz_cov = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            covarianza = np.sum((stocks.iloc[:, i] - retornos[i]) *
                                (stocks.iloc[:, j] - retornos[j])) / m
            matriz_cov[i, j] = covarianza
    
    return retornos.values, matriz_cov


In [ ]:
# -------------------------------------------------
# 4. Retorno y varianza esperados de un portafolio
# -------------------------------------------------
def portafolio(retornos, matriz_cov, X):
    R_p = np.dot(retornos, X)
    S_p = np.dot(X, np.dot(matriz_cov, X))
    return R_p, S_p

In [ ]:
# -------------------------------------------------
# 5. Portafolio de mínima varianza
# -------------------------------------------------
def portafolio_min_var(retornos, matriz_cov):
    n = len(matriz_cov)
    U = np.ones(n)
    S_inv = np.linalg.inv(matriz_cov)
    num = np.dot(U, S_inv)
    den = np.dot(num, U.T)
    X_min = num / den
    
    R_min, S_min = portafolio(retornos, matriz_cov, X_min)
    
    df = {"Retorno": R_min, "Varianza": S_min}
    for i, peso in enumerate(X_min):
        df[f"Peso_{i+1}"] = peso
    
    return df

In [ ]:
# -------------------------------------------------
# 6. Frontera eficiente
# -------------------------------------------------
def get_frontera_eficiente(stocks: pd.DataFrame):
    retornos, matriz_cov = valores_muestrales(stocks)
    
    P_min = portafolio_min_var(retornos, matriz_cov)
    R_min = P_min["Retorno"]
    
    Rf = np.arange(-2, R_min, 0.0001)
    dfportfolios = []
    
    for r in Rf:
        R_mod = retornos - r
        A = np.linalg.solve(matriz_cov, R_mod)
        X = A / np.sum(A)
        R, S = portafolio(retornos, matriz_cov, X)
        row = {"Retorno": R, "Varianza": S}
        for i, peso in enumerate(X):
            row[f"Peso_{i+1}"] = peso
        dfportfolios.append(row)
    
    return pd.DataFrame(dfportfolios)

In [ ]:
# -------------------------------------------------
# 7. Graficar Curva de Markowitz
# -------------------------------------------------
def fplot(stocks, weight_min, weight_max, ret_max=None, sigma_max=None):
    retornos, matriz_cov = valores_muestrales(stocks)
    
    n = 20000
    m = len(retornos)
    vR = []
    vS = []
    
    for _ in range(n):
        X = np.random.uniform(weight_min, weight_max, m)
        X = X / np.sum(X)
        R, S = portafolio(retornos, matriz_cov, X)
        vR.append(R)
        vS.append(S)
    
    plt.figure(figsize=(10,6))
    plt.scatter(vS, vR, c=vR, cmap="viridis", s=5)
    plt.xlabel("Varianza")
    plt.ylabel("Retorno")
    plt.title("Frontera de Markowitz")
    plt.colorbar(label="Retorno")
    plt.show()

In [ ]:

def port_return(weights, mu):
    return np.dot(weights, mu)


def port_variance(weights, cov):
    return float(weights.T.dot(cov).dot(weights))


def port_std(weights, cov):
    return np.sqrt(port_variance(weights, cov))

# Restricción: suma de pesos = 1
cons = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Límite: pesos entre -1 y 1 (permitir cortos hasta suponerlo)
bounds = None

# %% [markdown]
## 7.1 Portafolio de mínima varianza global (GMV)

# %%
def gmv_weights(cov):
    inv_cov = np.linalg.inv(cov)
    ones = np.ones(inv_cov.shape[0])
    w = inv_cov.dot(ones) / ones.dot(inv_cov).dot(ones)
    return w

# %% [markdown]
## 7.2 Frontera eficiente mediante optimización

# %%
def efficient_portfolio_for_return(target_return, mu, cov):
    n = len(mu)
    # Punto inicial: pesos iguales
    x0 = np.repeat(1/n, n)

    # Restricciones: suma de pesos =1 y retorno objetivo
    cons = (
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w: np.dot(w, mu) - target_return}
    )
    # Permitir pesos sin restricción de signo por defecto
    bounds = tuple((None, None) for _ in range(n))

    result = minimize(lambda w: w.T.dot(cov).dot(w), x0, method='SLSQP', constraints=cons, bounds=bounds)
    return result.x

# %% [markdown]
## 7.3 Construir la frontera eficiente

# %%
def efficient_frontier(mu, cov, returns_grid):
    weights = []
    risks = []
    rets = []
    for r in returns_grid:
        w = efficient_portfolio_for_return(r, mu, cov)
        weights.append(w)
        risks.append(port_std(w, cov))
        rets.append(np.dot(w, mu))
    return np.array(weights), np.array(rets), np.array(risks)

# %% [markdown]
## 7.4 Portafolio tangente (máxima Sharpe) dado r_f

# %%
def tangency_portfolio(mu, cov, r_f=0.0):
    # pesonal proporcional a Sigma^{-1}(mu - r_f 1)
    excess = mu - r_f * np.ones_like(mu)
    inv_cov = np.linalg.inv(cov)
    x = inv_cov.dot(excess)
    w = x / np.sum(x)
    return w

# %% [markdown]
## 8. Ejemplo de uso con datos sintéticos

# %%
# Definimos 4 activos con medias y covarianza sintética
np.random.seed(42)

n_assets = 4
mu = np.array([0.10, 0.12, 0.08, 0.11])  # rendimientos esperados anuales
# Construimos una matriz de covarianza razonable
A = np.array([[0.10, 0.08, 0.02, 0.04],
              [0.08, 0.20, 0.06, 0.07],
              [0.02, 0.06, 0.15, 0.03],
              [0.04, 0.07, 0.03, 0.12]])
# Convertimos a covarianza anual asumida (si fueran retornos diarios habría que ajustar)
cov = A * 0.25  # sólo para escalar

# %%
# Cálculo del portafolio GMV
w_gmv = gmv_weights(cov)
ret_gmv = port_return(w_gmv, mu)
risk_gmv = port_std(w_gmv, cov)

print('Pesos GMV:', np.round(w_gmv, 4))
print('Retorno esperado GMV:', np.round(ret_gmv, 4))
print('Riesgo (std) GMV:', np.round(risk_gmv, 4))

# %%
# Calculamos la frontera eficiente
returns_grid = np.linspace(min(mu)*0.9, max(mu)*1.1, 50)
weights_grid, rets_grid, risks_grid = efficient_frontier(mu, cov, returns_grid)

# %%
# Portafolio tangente asumiendo rf = 0.02 (2% anual)
rf = 0.02
w_tan = tangency_portfolio(mu, cov, rf)
ret_tan = port_return(w_tan, mu)
risk_tan = port_std(w_tan, cov)

print('\nPesos portafolio tangente:', np.round(w_tan, 4))
print('Retorno esperado tangente:', np.round(ret_tan, 4))
print('Riesgo (std) tangente:', np.round(risk_tan, 4))

# %% [markdown]
## 9. Gráfica de la frontera eficiente

# %%
plt.figure(figsize=(8,6))
plt.plot(risks_grid, rets_grid, label='Frontera eficiente')
plt.scatter(risk_gmv, ret_gmv, marker='*', color='red', s=100, label='GMV')
plt.scatter(risk_tan, ret_tan, marker='D', color='green', s=80, label='Tangente')
plt.xlabel('Riesgo (desviación estándar)')
plt.ylabel('Retorno esperado')
plt.title('Frontera eficiente - ejemplo sintético')
plt.legend()
plt.grid(True)
plt.show()
